In [4]:
import numpy as np
import scipy.sparse.linalg as slg

Solving multistate sequence optomization problem.


Given:

Number of sequences of states with weights. Example:

```
[A:0.5, B:0.3] -> [C:0.1, A:3.0] -> [B:1.0, C:0.5]

[A:0.3, B:0.6] -> [C:0.2, A:2.0] -> [B:0.4, C:0.7]
```

Output:

Matrix of transition weights:

$W = \{w_{X \rightarrow Y}\}$

In [25]:
X1 = np.matrix([
    [1,  0,  0,   0,   0  ], # init
    [0,  1,  0,   0,   0  ], # A
    [0,  0,  0.5, 0.5, 1  ], # B
    [0,  0,  0.5, 1,   0.5]  # C
])

X2 = np.matrix([
    [1,  0,  0,   0,   0   ], # init
    [0,  1,  0,   0,   0   ], # A
    [0,  0,  0.6, 0.55, 1  ], # B
    [0,  0,  0.4, 0.95, 0.5]  # C
])

Xs = [X1, X2]

In [15]:
def get_coef(xi, derive_point, coef_point):
    second = coef_point
    first = derive_point
    return np.dot(xi[second], xi[first].transpose())[0,0] * 2

def get_B(xi, x, dp):
    return 2 * np.dot(xi[dp[0]], x[dp[1]].transpose())[0,0]

$\sum_{X \in Train} || X_{init} \times W^T - X || \rightarrow min $

In [22]:
def find_matrixes(xi, x):
    N = xi.shape[0]
    coefs = np.zeros([N,N])
    Bs =  np.zeros([N,N])
    for i1 in range(0,N):
        for i2 in range(0,N):
            coefs[i1,i2] = get_coef(xi, i1, i2)
            Bs[i1,i2] = get_B(xi, x, (i2, i1))
            #print((i2, i1))
    
    return (coefs, Bs)
    
    #Bs[eq_num] = get_B(xi, x, (i,j))
    #print(coefs)
    #print(Bs)
def calc_weights(coefs, Bs):
    N = Bs.shape[0]
    W = []
    for i in range(0, N):
        W.append(np.linalg.solve(coefs, Bs[i]))#slg.spsolve(coefs, Bs)
    W = np.matrix(W).transpose()
    #print(W)
    return W

def learn_weights(Xs):
    N = Xs[0].shape[0]
    coefs = np.zeros([N,N])
    Bs =  np.zeros([N,N])
    for X in Xs:
        xi = X[:,:-1]
        x = X[:,1:]
        c, b = find_matrixes(xi, x)
        coefs += c
        Bs += b
    return calc_weights(coefs, Bs)
        

In [32]:
W = learn_weights(Xs)
W

matrix([[ 0.        ,  1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.55      ,  0.45      ],
        [ 0.        ,  0.        ,  0.2723869 ,  2.20343214],
        [ 0.        ,  0.        ,  0.86833073, -0.63900156]])

In [31]:
s = 0
for X in Xs:
    xi = X[:,:-1]
    x = X[:,1:]
    err = np.dot(W.transpose(), xi) - x
    s += np.multiply(err, err).sum()
print("error: {0}".format(s))

error: 0.0905226209048362
